In [1]:
import pandas as pd
import rasterio
import geopandas as gpd
import os
import rasterstats


In [2]:
#get electricity consumption for lau units
elec_lau1 = pd.read_excel(snakemake.input["elec_lau1"],
    sheet_name=str(snakemake.params.year) , header=[0, 1, 2])

elec_lau1 = elec_lau1.set_index(
    ('Sub-national electricity sales and numbers of customers, 2014 (1)(4)', 'Unnamed: 2_level_1', 'LA Code')
)[('Sales (GWh)', 'All')].dropna()
elec_lau1.index = elec_lau1.index.rename('LA Code')

# Shapefile of the LAU1 units
uk_lau1_units = gpd.read_file(snakemake.input["lau1_units"])
uk_lau1_units = uk_lau1_units.set_index('LAD19CD')[['LAD19NM', 'geometry']]
uk_lau1_units.index = uk_lau1_units.index.rename('LA Code')

# join shapefile and electricity demand
elec_lau1=gpd.GeoDataFrame(elec_lau1.join(uk_lau1_units).dropna(), geometry='geometry')

#import grided population data
with rasterio.open(snakemake.input["pop_uk"]) as pop_grid:
        array = pop_grid.read(1)
        crs = pop_grid.crs
        affine = pop_grid.transform
        
#importing Calliope model zones 
calliope_zones=gpd.read_file(snakemake.input["calliope_zones"])


In [3]:
#overlay
lau_zones = gpd.overlay(calliope_zones.to_crs(crs), elec_lau1.to_crs(crs), how='intersection', keep_geom_type=False)

#calculate population per snippet
pop = rasterstats.zonal_stats(lau_zones.to_crs(crs), array, affine=affine, stats='sum', nodata=0)
lau_zones['population'] = [i['sum'] for i in pop]

# weighted average according to population
lau_zones = lau_zones.set_index(['Name_1', 'LAD19NM'], drop=False)
weights = lau_zones.population.div(lau_zones.sum(level='LAD19NM').population)
lau_zones['Total consumption']=lau_zones['Total consumption']*weights
elec_zones=lau_zones.groupby(level='Name_1').sum()
elec_zones=elec_zones['Total consumption']

In [4]:
#get load profile 
load_profile_all= pd.read_csv(snakemake.input["load_profile"], header=0, index_col=0)
load_profile_all.index=pd.to_datetime(load_profile_all.index).tz_localize(None)
#choose year
load_profile=load_profile_all.loc[str(snakemake.params.year),'GB_GBN_load_actual_entsoe_transparency']

#normalise based on anual load
load_profile=load_profile/load_profile.sum()


In [6]:
#create demand dataframe
elec_demand=pd.concat([load_profile] * 20, axis=1)
elec_demand.columns=elec_zones.index
elec_demand=elec_demand.mul(elec_zones)
elec_demand=elec_demand*-1
elec_demand['EAST_ANGLIA']=0
elec_demand['HORNSEA']=0
elec_demand.index=pd.to_datetime(elec_demand.index)

#export to csv
elec_demand.to_csv(snakemake.output[0])